#### Carregando biblioteca de funções Turing

In [7]:
id_empresa = '1022'
nm_bucket = 'turing-bkt-treinamentos-prod'
%run -i '/home/.Turing/TuringCredentialsAccess.py3'
%run -i '/home/.Turing/TuringLabFunctions.py3'

-------------------- LabTuring ---------------------------
----------- Exploração de Dados e Modelagem --------------
------ Funções carregadas em memória com sucesso ---------
--------- Data da última atualização: --------------------
--------------- 10/11/2019 -------------------------------
----------------------------------------------------------


#### Mapear bibliotecas Python necessárias

In [8]:
import pandas as pd
import numpy as np

#### Gerando credenciais para acesso aos dados do projeto

In [9]:
S3fs,S3session,S3client,S3resource = TuringUsersCredentialsControl(id_empresa)

#### Parâmetros do Projeto

In [17]:
#---------- Definição dos Parâmetros do projeto ---------------------------------

# Nome da variável resposta
targetname = 'TARGET'

# Nome da ABT FULL
nm_file_desenv_orig = 'ABT_V01_BJ.csv'

# Nome da tabela de TREINAMENTO
nm_s3_file_train = 'ABT_V01_BJ_DESENV.csv'

# Nome da tabela de TESTE (OOS)
nm_s3_file_test = 'ABT_V01_BJ_VALID.csv'

#### Ler tabela de modelagem (ABT) 

In [18]:
# Lendo tabela de desenvolvimento original do bucket da POC
path_file = 'Projetos/FLAT_TABLES/'+str(nm_file_desenv_orig)
df_00 = TuringReadS3CSVFile(nm_bucket,path_file,sep=',')
df_00.shape

(307511, 150)

In [19]:
df_00.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,AVG_AMT_CREDIT_SUM_OVERDUE,MIN_DAYS_CREDIT_UPDATE,MAX_DAYS_CREDIT_UPDATE,AVG_DAYS_CREDIT_UPDATE,MIN_CNT_CREDIT_PROLONG,MAX_CNT_CREDIT_PROLONG,AVG_CNT_CREDIT_PROLONG,MIN_AMT_ANNUITY,MAX_AMT_ANNUITY,AVG_AMT_ANNUITY
0,100274,0,0,308250.0,1305000.0,38281.5,1305000.0,0.026392,-15612,-1106,...,0.0,-42.0,-42.0,-42.0,0.0,0.0,0.0,NaN,NaN,NaN
1,100800,0,0,126000.0,698517.0,23215.5,603000.0,0.018634,-21314,365243,...,0.0,-43.0,-43.0,-43.0,0.0,0.0,0.0,NaN,NaN,NaN
2,100884,0,0,157500.0,327024.0,21420.0,270000.0,0.018029,-10960,-1091,...,0.0,-14.0,-13.0,-13.5,0.0,0.0,0.0,NaN,NaN,NaN
3,100986,0,0,180000.0,640080.0,29970.0,450000.0,0.010147,-17231,-3683,...,0.0,-30.0,-30.0,-30.0,0.0,0.0,0.0,NaN,NaN,NaN
4,101094,0,0,112500.0,819432.0,23958.0,684000.0,0.028663,-23643,-3326,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Separar a tabela de modelagem em duas (tabela de desenvolvimento e tabela de teste/validação)
    - Neste caso estamos gerando validação out-of-sample

In [20]:
df_desenv,df_valid = TuringHoldOut(df_00,targetname,pct_test=0.3,tseed=123456)
df_desenv.shape,df_valid.shape

((215257, 150), (92254, 150))

#### Salvar tabelas para autoML Turing

In [21]:
s3_path_desenv = 'turing-bkt-treinamentos-prod/Projetos/FLAT_TABLES/'
TuringWriteS3CSVFile(df_desenv,s3_path_desenv,nm_s3_file_train,S3fs,delimiter=',')

s3_path_valida = 'turing-bkt-treinamentos-prod/Projetos/SCORE_BATCH/'
TuringWriteS3CSVFile(df_valid,s3_path_valida,nm_s3_file_test,S3fs,delimiter=',')

encoding=latin-1
encoding=latin-1


'Arquivo Salvo Com Sucesso'

### Verificar no S3 e no Timeline do Turing se os arquivos foram Salvos com Sucesso no local correto

www.labturing.com.br